# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [651]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3
import os


current_directory = os.getcwd()
print(current_directory)

# loading data from python.db database
con = sqlite3.connect(r'./data/python.db')

# import 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql('SELECT * FROM school', con)
student_data = pd.read_sql('SELECT * FROM student', con)


/Users/shaunkumar/Desktop/DS311/Python-HW-Repository/Lab_Assignment


In [653]:
# join the two tables into a single dataframe
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [656]:
# Create a District Summary
district_summary = school_data_complete.groupby('type').agg(
    student_count=('Student ID', 'count'),              # Count of students
    school_count=('School ID', 'nunique'),              # Count of unique schools
    total_budget=('budget', 'sum'),
    average_math_score=('math_score', 'mean'),
    average_reading_score=('reading_score', 'mean'),
)
district_summary['overall_average_score'] = (district_summary['average_math_score'] + district_summary['average_reading_score']) / 2

passing_students_math = school_data_complete[school_data_complete['math_score'] >= 70]
passing_students_math_count = passing_students_math.groupby('type')['Student ID'].count()
district_summary['passing_percentage_math'] = (passing_students_math_count / district_summary['student_count']) * 100

passing_students_reading = school_data_complete[school_data_complete['reading_score'] >= 70]
passing_students_reading_count = passing_students_reading.groupby('type')['Student ID'].count()
district_summary['passing_percentage_reading'] = (passing_students_reading_count / district_summary['student_count']) * 100

print(district_summary)

          student_count  school_count  total_budget  average_math_score  \
type                                                                      
Charter           12194             8   12493275585           83.406183   
District          26976             7   70439053973           76.987026   

          average_reading_score  overall_average_score  \
type                                                     
Charter               83.902821              83.654502   
District              80.962485              78.974755   

          passing_percentage_math  passing_percentage_reading  
type                                                           
Charter                 93.701821                   96.645891  
District                66.518387                   80.905249  


In [658]:
# Total number of schools
total_schools = school_data['School ID'].nunique()  

# Print the result
print(f"Total number of schools is: {total_schools}")



Total number of schools is: 15


In [660]:
# Total number of students
total_students = student_data['Student ID'].nunique()  

# Print the result
print(f"Total number of schools is: {total_students}")


Total number of schools is: 39170


In [662]:
# Total budget
total_budget = school_data['budget'].sum()  

# Print the result
print(f"Total budget of the district is: ${total_budget}")



Total budget of the district is: $24649428


In [664]:
# Average math score
avg_math_score = student_data['math_score'].mean()  

# Print the result
print(f"Average Math Score of the district is: {avg_math_score}")




Average Math Score of the district is: 78.98537145774827


In [666]:
# Average reading score
avg_reading_score = student_data['reading_score'].mean()  

# Print the result
print(f"Average Reading Score of the district is: {avg_reading_score}")



Average Reading Score of the district is: 81.87784018381414


In [668]:
# Overall average score
overall_avg_score = (avg_math_score + avg_reading_score) / 2

# Print the result
print(f"Overall Average Score of the district is: {overall_avg_score}")



Overall Average Score of the district is: 80.43160582078121


In [670]:
# Percentage of passing math (70 or greater)
# Count the number of students who have a math score of 70 or greater
passing_students_math = student_data[student_data['math_score'] >= 70]

# Calculate the number of passing students
num_passing_math_students = passing_students_math.shape[0]

# Calculate the total number of students
total_students = student_data.shape[0]

#Calculate the percentage of passing students
passing_math_percentage = (num_passing_math_students / total_students) * 100

# Print the result
print(f"Percentage of Students Passing Math in the District is: {passing_math_percentage}%")

Percentage of Students Passing Math in the District is: 74.9808526933878%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [676]:
#  Sort and display the top five schools in overall passing rate
# Calculate the passing percentage for math and reading for each student
student_data['passing_math'] = student_data['math_score'] >= 70
student_data['passing_reading'] = student_data['reading_score'] >= 70

# Calculate overall passing rate by combining math and reading passing rates
student_data['overall_passing'] = (student_data['passing_math'] & student_data['passing_reading'])

# Group by 'school_name' and calculate the overall passing percentage for each school
school_passing_rates = student_data.groupby('school_name').agg(
    total_students=('Student ID', 'count'),
    passing_students=('overall_passing', 'sum')
)

# Calculate the overall passing percentage for each school
school_passing_rates['passing_percentage'] = (school_passing_rates['passing_students'] / school_passing_rates['total_students']) * 100

# Sort the schools by passing percentage in descending order and display the top 5
top_5_schools = school_passing_rates.sort_values(by='passing_percentage', ascending=False).head(5)

# Print the top 5 schools based on overall passing rate
display(top_5_schools[['passing_percentage']])


,passing_percentage
school_name,
Cabrera High School,91.334769
Thomas High School,90.948012
Griffin High School,90.599455
Wilson High School,90.582567
Pena High School,90.540541


In [678]:
# Calculate the total budget for each school
total_school_budgets = school_data.groupby('school_name')['budget'].sum()

# Print the total budgets for each school
print(total_school_budgets)


school_name
Bailey High School       3124928
Cabrera High School      1081356
Figueroa High School     1884411
Ford High School         1763916
Griffin High School       917500
Hernandez High School    3022020
Holden High School        248087
Huang High School        1910635
Johnson High School      3094650
Pena High School          585858
Rodriguez High School    2547363
Shelton High School      1056600
Thomas High School       1043130
Wilson High School       1319574
Wright High School       1049400
Name: budget, dtype: int64


In [680]:
# Calculate per student budget

# Calculate the total number of students in each school
students_per_school = school_data.groupby('school_name')['size'].sum()

# Calculate the per student budget for each school
per_student_budget = total_school_budgets / students_per_school

# Print the per student budget for each school
print(per_student_budget)




school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64


In [682]:
# Cacluate the avg math and reading score
# Calculate the average math and average reading scores for each school
average_scores = student_data.groupby('school_name').agg(
    average_math_score=('math_score', 'mean'),
    average_reading_score=('reading_score', 'mean')
)

# Print the average scores for each school
display(average_scores)




,average_math_score,average_reading_score
school_name,,
Bailey High School,77.048432,81.033963
Cabrera High School,83.061895,83.975780
Figueroa High School,76.711767,81.158020
Ford High School,77.102592,80.746258
Griffin High School,83.351499,83.816757
Hernandez High School,77.289752,80.934412
Holden High School,83.803279,83.814988
Huang High School,76.629414,81.182722
Johnson High School,77.072464,80.966394


#### Find the passing rate for math and reading (above 70 points)

In [685]:
# Find the total counts of math result
math_score_counts = student_data.groupby('school_name')['math_score'].count()

# Find the counts for math result in each school that pass 70 or higher
passing_math = student_data[student_data['math_score'] >= 70]
passing_math_counts = passing_math.groupby('school_name')['math_score'].count()

# Calculate the math passing rate
math_passing_rate = (passing_math_counts / math_score_counts) * 100

print(math_passing_rate)

school_name
Bailey High School       66.680064
Cabrera High School      94.133477
Figueroa High School     65.988471
Ford High School         68.309602
Griffin High School      93.392371
Hernandez High School    66.752967
Holden High School       92.505855
Huang High School        65.683922
Johnson High School      66.057551
Pena High School         94.594595
Rodriguez High School    66.366592
Shelton High School      93.867121
Thomas High School       93.272171
Wilson High School       93.867718
Wright High School       93.333333
Name: math_score, dtype: float64


In [687]:
# Find the total counts of read result
reading_score_counts = student_data.groupby('school_name')['reading_score'].count()


# Find the counts for read result in each school that pass 70 or higher
passing_reading = student_data[student_data['reading_score'] >= 70]
passing_reading_counts = passing_reading.groupby('school_name')['reading_score'].count()

# Calculate the read passing rate
reading_passing_rate = (passing_reading_counts / reading_score_counts) * 100

print(reading_passing_rate)


school_name
Bailey High School       81.933280
Cabrera High School      97.039828
Figueroa High School     80.739234
Ford High School         79.299014
Griffin High School      97.138965
Hernandez High School    80.862999
Holden High School       96.252927
Huang High School        81.316421
Johnson High School      81.222432
Pena High School         95.945946
Rodriguez High School    80.220055
Shelton High School      95.854628
Thomas High School       97.308869
Wilson High School       96.539641
Wright High School       96.611111
Name: reading_score, dtype: float64


In [689]:
# Calculate the overall passing rate (average of the math and reading passing rate)
overall_passing_rate = (math_passing_rate + reading_passing_rate) / 2
print(overall_passing_rate)


school_name
Bailey High School       74.306672
Cabrera High School      95.586652
Figueroa High School     73.363852
Ford High School         73.804308
Griffin High School      95.265668
Hernandez High School    73.807983
Holden High School       94.379391
Huang High School        73.500171
Johnson High School      73.639992
Pena High School         95.270270
Rodriguez High School    73.293323
Shelton High School      94.860875
Thomas High School       95.290520
Wilson High School       95.203679
Wright High School       94.972222
dtype: float64


### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [693]:
#  Sort and display the worst five schools in overall passing rate
worst_five_schools = overall_passing_rate.sort_values(ascending=True).head(5)
print("Worst Five Schools in Overall Passing Rate:")
print(worst_five_schools)


Worst Five Schools in Overall Passing Rate:
school_name
Rodriguez High School    73.293323
Figueroa High School     73.363852
Huang High School        73.500171
Johnson High School      73.639992
Ford High School         73.804308
dtype: float64


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [697]:
# Create table that lists the average math score for each school of each grade level.
average_math_by_grade = student_data.groupby(['school_name', 'grade'])['math_score'].mean().unstack()

# Reorder columns to ensure they are sorted from 9th to 12th
average_math_by_grade = average_math_by_grade[['9th', '10th', '11th', '12th']]

# Printing results to verify 
print("Average Math Score by Grade Level for Each School:")
display(average_math_by_grade)



Average Math Score by Grade Level for Each School:


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [699]:
# Calculate the average math score for 9th grade in each school
ninth_grade_students = student_data[student_data['grade'] == '9th']
average_math_9th_grade = ninth_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 9th Grade in Each School:")
print(average_math_9th_grade)


Average Math Score for 9th Grade in Each School:
school_name
Bailey High School       77.083676
Cabrera High School      83.094697
Figueroa High School     76.403037
Ford High School         77.361345
Griffin High School      82.044010
Hernandez High School    77.438495
Holden High School       83.787402
Huang High School        77.027251
Johnson High School      77.187857
Pena High School         83.625455
Rodriguez High School    76.859966
Shelton High School      83.420755
Thomas High School       83.590022
Wilson High School       83.085578
Wright High School       83.264706
Name: math_score, dtype: float64


In [701]:
# Calculate the average math score for 10th grade in each school
tenth_grade_students = student_data[student_data['grade'] == '10th']
average_math_10th_grade = tenth_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 10th Grade in Each School:")
print(average_math_10th_grade)


Average Math Score for 10th Grade in Each School:
school_name
Bailey High School       76.996772
Cabrera High School      83.154506
Figueroa High School     76.539974
Ford High School         77.672316
Griffin High School      84.229064
Hernandez High School    77.337408
Holden High School       83.429825
Huang High School        75.908735
Johnson High School      76.691117
Pena High School         83.372000
Rodriguez High School    76.612500
Shelton High School      82.917411
Thomas High School       83.087886
Wilson High School       83.724422
Wright High School       84.010288
Name: math_score, dtype: float64


In [703]:
# Calculate the average math score for 11th grade in each school
eleventh_grade_students = student_data[student_data['grade'] == '11th']
average_math_11th_grade = eleventh_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 11th Grade in Each School:")
print(average_math_11th_grade)


Average Math Score for 11th Grade in Each School:
school_name
Bailey High School       77.515588
Cabrera High School      82.765560
Figueroa High School     76.884344
Ford High School         76.918058
Griffin High School      83.842105
Hernandez High School    77.136029
Holden High School       85.000000
Huang High School        76.446602
Johnson High School      77.491653
Pena High School         84.328125
Rodriguez High School    76.395626
Shelton High School      83.383495
Thomas High School       83.498795
Wilson High School       83.195326
Wright High School       83.836782
Name: math_score, dtype: float64


In [705]:
# Calculate the average math score for 12th grade in each school
twelfth_grade_students = student_data[student_data['grade'] == '12th']
average_math_12th_grade = twelfth_grade_students.groupby('school_name')['math_score'].mean()

# Print
print("Average Math Score for 12th Grade in Each School:")
print(average_math_12th_grade)


Average Math Score for 12th Grade in Each School:
school_name
Bailey High School       76.492218
Cabrera High School      83.277487
Figueroa High School     77.151369
Ford High School         76.179963
Griffin High School      83.356164
Hernandez High School    77.186567
Holden High School       82.855422
Huang High School        77.225641
Johnson High School      76.863248
Pena High School         84.121547
Rodriguez High School    77.690748
Shelton High School      83.778976
Thomas High School       83.497041
Wilson High School       83.035794
Wright High School       83.644986
Name: math_score, dtype: float64


### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [709]:
# Create table that lists the average reading score for each school of each grade level.
average_reading_by_grade = student_data.groupby(['school_name', 'grade'])['reading_score'].mean().unstack()

# Reorder columns to ensure they are sorted from 9th to 12th
average_reading_by_grade = average_reading_by_grade[['9th', '10th', '11th', '12th']]

# Printing results to verify 
print("Average Reading Score by Grade Level for Each School:")
display(average_reading_by_grade)


Average Reading Score by Grade Level for Each School:


grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [710]:
# Calculate the average reading score for 9th grade in each school
average_reading_9th_grade = ninth_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 9th Grade in Each School:")
print(average_reading_9th_grade)


Average Reading Score for 9th Grade in Each School:
school_name
Bailey High School       81.303155
Cabrera High School      83.676136
Figueroa High School     81.198598
Ford High School         80.632653
Griffin High School      83.369193
Hernandez High School    80.866860
Holden High School       83.677165
Huang High School        81.290284
Johnson High School      81.260714
Pena High School         83.807273
Rodriguez High School    80.993127
Shelton High School      84.122642
Thomas High School       83.728850
Wilson High School       83.939778
Wright High School       83.833333
Name: reading_score, dtype: float64


In [713]:
# Calculate the average reading score for 10th grade in each school
average_reading_10th_grade = tenth_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 10th Grade in Each School:")
print(average_reading_10th_grade)


Average Reading Score for 10th Grade in Each School:
school_name
Bailey High School       80.907183
Cabrera High School      84.253219
Figueroa High School     81.408912
Ford High School         81.262712
Griffin High School      83.706897
Hernandez High School    80.660147
Holden High School       83.324561
Huang High School        81.512386
Johnson High School      80.773431
Pena High School         83.612000
Rodriguez High School    80.629808
Shelton High School      83.441964
Thomas High School       84.254157
Wilson High School       84.021452
Wright High School       83.812757
Name: reading_score, dtype: float64


In [715]:
# Calculate the average reading score for 11th grade in each school
average_reading_11th_grade = eleventh_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 11th Grade in Each School:")
print(average_reading_11th_grade)


Average Reading Score for 11th Grade in Each School:
school_name
Bailey High School       80.945643
Cabrera High School      83.788382
Figueroa High School     80.640339
Ford High School         80.403642
Griffin High School      84.288089
Hernandez High School    81.396140
Holden High School       83.815534
Huang High School        81.417476
Johnson High School      80.616027
Pena High School         84.335938
Rodriguez High School    80.864811
Shelton High School      84.373786
Thomas High School       83.585542
Wilson High School       83.764608
Wright High School       84.156322
Name: reading_score, dtype: float64


In [717]:
# Calculate the average reading score for 12th grade in each school
average_reading_12th_grade = twelfth_grade_students.groupby('school_name')['reading_score'].mean()

# Print
print("Average Reading Score for 12th Grade in Each School:")
print(average_reading_12th_grade)


Average Reading Score for 12th Grade in Each School:
school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [721]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 630, 645, float('inf')]
group_names = ["<$585", "$585-630", "$630-645", ">$645"]

In [727]:
# Create a new column to show budget per student in each row
school_data['spending_range'] = pd.cut(school_data['spending_per_student'], bins=spending_bins, labels=group_names)
avg_spending_by_range = school_data.groupby('spending_range')['spending_per_student'].mean().reset_index(name='avg_budget_per_student')

# Print the final grouped data
display(avg_spending_by_range[['avg_budget_per_student']])


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/12972625.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_spending_by_range = school_data.groupby('spending_range')['spending_per_student'].mean().reset_index(name='avg_budget_per_student')


,avg_budget_per_student
0,581.000000
1,615.500000
2,639.500000
3,652.333333


In [731]:
# Create a new column to show budget per student in each row

school_data['spending_per_student'] = school_data['budget'] / school_data['size']
avg_spending_by_range = school_data.groupby('spending_range')['spending_per_student'].mean().reset_index(name='avg_budget_per_student')



/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/796374028.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_spending_by_range = school_data.groupby('spending_range')['spending_per_student'].mean().reset_index(name='avg_budget_per_student')


In [733]:
# Create a new column to define the spending ranges per student


# Print the final grouped data
display(avg_spending_by_range)


,spending_range,avg_budget_per_student
0,<$585,581.000000
1,$585-630,615.500000
2,$630-645,639.500000
3,>$645,652.333333


In [735]:
# Calculate the average math score within each spending range

# Merge student_data with school_data based on 'school_name'
merged_data = pd.merge(student_data[['school_name', 'math_score', 'reading_score']], 
                       school_data[['school_name', 'spending_range']], 
                       on='school_name', how='left')

# Calculate the average math score for each spending range
average_math_by_spending = merged_data.groupby('spending_range')['math_score'].mean().reset_index(name='average_math_score_in_range')

# Merge the average spending and average math score data
final_grouped_data = pd.merge(avg_spending_by_range, average_math_by_spending, on='spending_range')

# Display the final table with average spending per student and average math score in each spending range
display(final_grouped_data[['spending_range', 'avg_budget_per_student', 'average_math_score_in_range']])


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/745075440.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_math_by_spending = merged_data.groupby('spending_range')['math_score'].mean().reset_index(name='average_math_score_in_range')


,spending_range,avg_budget_per_student,average_math_score_in_range
0,<$585,581.000000,83.363065
1,$585-630,615.500000,79.982873
2,$630-645,639.500000,77.821056
3,>$645,652.333333,77.049297


In [737]:
# Calculate the percentage passing rate for math in each spending range

# Calculate the percentage of students passing math (>= 70) in each spending range
merged_data['passing_math'] = merged_data['math_score'] >= 70

# Group by spending_range, calculate the passing rate for math
passing_math_rate = merged_data.groupby('spending_range').agg(
    total_students=('math_score', 'count'),
    passing_students=('passing_math', 'sum')
).reset_index()

# Calculate the passing rate for math as a percentage
passing_math_rate['passing_math_percentage_in_range'] = (passing_math_rate['passing_students'] / passing_math_rate['total_students']) * 100

# Merge the results into a final DataFrame, using 'spending_range' to join
final_grouped_data = pd.merge(
    avg_spending_by_range, 
    average_math_by_spending, 
    on='spending_range', 
    how='left'
)

final_grouped_data = pd.merge(
    final_grouped_data, 
    passing_math_rate[['spending_range', 'passing_math_percentage_in_range']], 
    on='spending_range', 
    how='left'
)

# Display the final table with the aggregated results
display(final_grouped_data[['spending_range', 'avg_budget_per_student', 
                            'average_math_score_in_range', 'passing_math_percentage_in_range']])


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/596567602.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_math_rate = merged_data.groupby('spending_range').agg(


,spending_range,avg_budget_per_student,average_math_score_in_range,passing_math_percentage_in_range
0,<$585,581.000000,83.363065,93.702889
1,$585-630,615.500000,79.982873,79.109851
2,$630-645,639.500000,77.821056,70.623565
3,>$645,652.333333,77.049297,66.230813


In [739]:
# Calculate the percentage passing rate for reading in each spending range

# Calculate the percentage of students passing reading (>= 70) in each spending range
merged_data['passing_reading'] = merged_data['reading_score'] >= 70

# Group by spending_range, calculate the passing rate for reading
passing_reading_rate = merged_data.groupby('spending_range').agg(
    total_students=('reading_score', 'count'),
    passing_students=('passing_reading', 'sum')
).reset_index()

# Calculate the passing rate for reading as a percentage
passing_reading_rate['passing_reading_percentage_in_range'] = (passing_reading_rate['passing_students'] / passing_reading_rate['total_students']) * 100

# Now, merge the results into a final DataFrame, using 'spending_range' to join
final_grouped_data = pd.merge(avg_spending_by_range, average_math_by_spending, on='spending_range', how='left')

final_grouped_data = pd.merge(
    final_grouped_data, 
    passing_math_rate[['spending_range', 'passing_math_percentage_in_range']], 
    on='spending_range', how='left'
)

final_grouped_data = pd.merge(
    final_grouped_data, 
    passing_reading_rate[['spending_range', 'passing_reading_percentage_in_range']], 
    on='spending_range', how='left'
)

# Display the final table with the aggregated results
display(final_grouped_data[['spending_range', 'avg_budget_per_student', 
                            'average_math_score_in_range', 'passing_math_percentage_in_range', 
                            'passing_reading_percentage_in_range']])


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/1579310321.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_reading_rate = merged_data.groupby('spending_range').agg(


,spending_range,avg_budget_per_student,average_math_score_in_range,passing_math_percentage_in_range,passing_reading_percentage_in_range
0,<$585,581.000000,83.363065,93.702889,96.686558
1,$585-630,615.500000,79.982873,79.109851,88.513145
2,$630-645,639.500000,77.821056,70.623565,82.600247
3,>$645,652.333333,77.049297,66.230813,81.109397


In [741]:
# Calculate the percentage overall passing rate in each spending range

# Group by spending_range to calculate average math and reading scores
average_scores = merged_data.groupby('spending_range').agg(
    average_math_score=('math_score', 'mean'),
    average_reading_score=('reading_score', 'mean')
).reset_index()


# Calculate the passing rates as percentages for math and reading
passing_rates = merged_data.groupby('spending_range').agg(
    passing_math_percentage=('passing_math', 'mean'),
    passing_reading_percentage=('passing_reading', 'mean')
).reset_index()

# Convert passing rates from proportion to percentage
passing_rates['passing_math_percentage'] *= 100
passing_rates['passing_reading_percentage'] *= 100

# Merge the average scores and passing rates
performance_summary = pd.merge(average_scores, passing_rates, on='spending_range')

# Calculate the overall passing rate as the average of math and reading passing rates
performance_summary['overall_passing_rate'] = (
    (performance_summary['passing_math_percentage'] + performance_summary['passing_reading_percentage']) / 2
)

# Calculate average spending per student (avg_budget_per_student) for each spending range
avg_budget_per_student = school_data.groupby('spending_range')['spending_per_student'].mean().reset_index(name='avg_budget_per_student')

# Merge the performance summary with the average budget per student
final_grouped_data = pd.merge(performance_summary, avg_budget_per_student, on='spending_range', how='left')

# Calculate passing rates for math
merged_data['passing_math'] = merged_data['math_score'] >= 70
passing_math_rate = merged_data.groupby('spending_range').agg(
    total_students=('math_score', 'count'),
    passing_students=('passing_math', 'sum')
).reset_index()

passing_math_rate['passing_math_percentage_in_range'] = (passing_math_rate['passing_students'] / passing_math_rate['total_students']) * 100

# Merge passing math rate back into the final data
final_grouped_data = pd.merge(final_grouped_data, passing_math_rate[['spending_range', 'passing_math_percentage_in_range']], 
                              on='spending_range', how='left')

# Display the final table with the aggregated results
display(final_grouped_data[['spending_range', 'avg_budget_per_student', 
                            'average_math_score', 'passing_math_percentage_in_range', 
                            'passing_reading_percentage', 'overall_passing_rate']])

/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/1412236327.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_scores = merged_data.groupby('spending_range').agg(
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/1412236327.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_rates = merged_data.groupby('spending_range').agg(
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/1412236327.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current b

,spending_range,avg_budget_per_student,average_math_score,passing_math_percentage_in_range,passing_reading_percentage,overall_passing_rate
0,<$585,581.000000,83.363065,93.702889,96.686558,95.194724
1,$585-630,615.500000,79.982873,79.109851,88.513145,83.811498
2,$630-645,639.500000,77.821056,70.623565,82.600247,76.611906
3,>$645,652.333333,77.049297,66.230813,81.109397,73.670105


### Scores by School Size

* Perform the same operations as above, based on school size.

In [745]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [747]:
# Create a new column for the bin groups
school_data['size_range'] = pd.cut(school_data['size'], bins=size_bins, labels=group_names)

# Print the updated table to verify the new column
display(school_data.groupby('size_range').size().reset_index(name='school_count'))

/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/3884248290.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(school_data.groupby('size_range').size().reset_index(name='school_count'))


,size_range,school_count
0,Small (<1000),2
1,Medium (1000-2000),5
2,Large (2000-5000),8


Look for the total count of test scores that pass 70% or higher




In [750]:
# math_pass_size

# Merge school_data and student_data on 'school_name'
merged_data = pd.merge(school_data, student_data, on='school_name')

# Create the 'size_range' column based on school size in the merged data
merged_data['size_range'] = pd.cut(merged_data['size'], bins=size_bins, labels=group_names)

# Create a new column 'math_pass_size' where 1 represents passing (math_score >= 70) and 0 represents failing
merged_data['math_pass_size'] = merged_data['math_score'] >= 70

# Count the total number of passing math scores for each school
passing_math_counts = merged_data.groupby('school_name')['math_pass_size'].sum().reset_index(name='passing_math_counts')

# Merge passing math counts with the size_range data
final_data = pd.merge(merged_data[['school_name', 'size_range']].drop_duplicates(), passing_math_counts, on='school_name')

# Group by size_range to get the total passing math counts for each size range
grouped_by_size_range = final_data.groupby('size_range')['passing_math_counts'].sum().reset_index()

# Print the final grouped table
display(grouped_by_size_range)


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/3240505658.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_size_range = final_data.groupby('size_range')['passing_math_counts'].sum().reset_index()


,size_range,passing_math_counts
0,Small (<1000),1305
1,Medium (1000-2000),7978
2,Large (2000-5000),20087


In [752]:
# read_pass_size

# Create the 'read_pass_size' column 
merged_data['read_pass_size'] = merged_data['reading_score'] >= 70

# Count the total number of passing reading scores for each school
passing_read_counts = merged_data.groupby('school_name')['read_pass_size'].sum().reset_index(name='passing_read_counts')

# Count the total number of passing math scores for each school (assuming 'passing_math_counts' already exists)
# If not, we can replicate the same logic as for reading scores.
passing_math_counts = merged_data.groupby('school_name')['math_pass_size'].sum().reset_index(name='passing_math_counts')

# Now, merge passing math and reading counts with the size_range data
final_data = pd.merge(merged_data[['school_name', 'size_range']].drop_duplicates(), 
                       passing_math_counts, on='school_name')
final_data = pd.merge(final_data, passing_read_counts, on='school_name')

# Group by size_range to get the total passing math and reading counts for each size range
grouped_by_size_range = final_data.groupby('size_range')[['passing_math_counts', 'passing_read_counts']].sum().reset_index()

# Print the final grouped table
display(grouped_by_size_range)

/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/2885624341.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_size_range = final_data.groupby('size_range')[['passing_math_counts', 'passing_read_counts']].sum().reset_index()


,size_range,passing_math_counts,passing_read_counts
0,Small (<1000),1305,1334
1,Medium (1000-2000),7978,8247
2,Large (2000-5000),20087,24029


In [754]:
# Calculate the overall passing rate for different school size

#Calculate the total number of students in each school size range
total_students_by_size = merged_data.groupby('size_range')['school_name'].count().reset_index(name='total_students')

# Calculate the total number of passing students (both math and reading) in each school
total_passing_by_size = merged_data.groupby('size_range').apply(
    lambda x: (x['math_pass_size'] + x['read_pass_size']).sum()).reset_index(name='total_passing_students')

# Merge the passing student counts with the total students for each size range
passing_rate_data = pd.merge(total_students_by_size, total_passing_by_size, on='size_range')

# Calculate the overall passing rate for each size range
passing_rate_data['overall_passing_rate'] = passing_rate_data['total_passing_students'] / passing_rate_data['total_students'] * 100

# Print the final data
display(passing_rate_data)



/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/3687487584.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_students_by_size = merged_data.groupby('size_range')['school_name'].count().reset_index(name='total_students')
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/3687487584.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_passing_by_size = merged_data.groupby('size_range').apply(
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/3687487584.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is de

,size_range,total_students,total_passing_students,overall_passing_rate
0,Small (<1000),1389,1387,99.856012
1,Medium (1000-2000),8522,8502,99.765313
2,Large (2000-5000),29259,27563,94.203493


### Scores by School Type

* Perform the same operations as above, based on school type.

In [758]:
# Create bins and groups, school type {'Charter', 'District'}
type_bins = [0, 2300, float('inf')]
group_names = ["Charter", "District"]

# Create a new column for the bin groups
school_data['school_type'] = pd.cut(school_data['size'], bins=type_bins, labels=group_names)

# Print the updated table to verify the new column
display(school_data.groupby('school_type').size().reset_index(name='school_count'))

/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/2870053191.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(school_data.groupby('school_type').size().reset_index(name='school_count'))


,school_type,school_count
0,Charter,8
1,District,7


Find counts of the passing 70 or higher score for the both test


In [761]:
# math pass size

# Merge school_data and student_data on 'school_name'
merged_data = pd.merge(school_data, student_data, on='school_name')

# Create the 'size_range' column based on school size in the merged data
merged_data['school_type'] = pd.cut(merged_data['size'], bins=type_bins, labels=group_names)

# Create a new column 'math_pass_size' where 1 represents passing (math_score >= 70) and 0 represents failing
merged_data['math_pass_size'] = merged_data['math_score'] >= 70

# Count the total number of passing math scores for each school
passing_math_counts = merged_data.groupby('school_name')['math_pass_size'].sum().reset_index(name='passing_math_counts')

# Merge passing math counts with the size_range data
final_data = pd.merge(merged_data[['school_name', 'school_type']].drop_duplicates(), passing_math_counts, on='school_name')

# Group by size_range to get the total passing math counts for each size range
grouped_by_size_range = final_data.groupby('school_type')['passing_math_counts'].sum().reset_index()

# Print the final grouped table
display(grouped_by_size_range)



/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/3426770494.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_size_range = final_data.groupby('school_type')['passing_math_counts'].sum().reset_index()


,school_type,passing_math_counts
0,Charter,11426
1,District,17944


In [763]:
# Merge school_data and student_data on 'school_name'
merged_data = pd.merge(school_data, student_data, on='school_name')

# Create the 'school_type' column based on the new bins
merged_data['school_type'] = pd.cut(merged_data['size'], bins=type_bins, labels=group_names)

# Create columns to represent passing scores (1 for pass, 0 for fail)
merged_data['math_pass_size'] = merged_data['math_score'] >= 70
merged_data['read_pass_size'] = merged_data['reading_score'] >= 70

# Count the total number of passing math and reading scores for each school
passing_counts = merged_data.groupby('school_name')[['math_pass_size', 'read_pass_size']].sum().reset_index()

# Merge passing counts with the school_type data
final_data = pd.merge(merged_data[['school_name', 'school_type']].drop_duplicates(), passing_counts, on='school_name')

# Group by school_type to get the total passing counts for both subjects
grouped_by_school_type = final_data.groupby('school_type')[['math_pass_size', 'read_pass_size']].sum().reset_index()

# Rename columns for clarity
grouped_by_school_type.rename(columns={'math_pass_size': 'math_pass_count', 'read_pass_size': 'read_pass_count'}, inplace=True)

# Print the final grouped table
display(grouped_by_school_type)


/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/2836063247.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_school_type = final_data.groupby('school_type')[['math_pass_size', 'read_pass_size']].sum().reset_index()


,school_type,math_pass_count,read_pass_count
0,Charter,11426,11785
1,District,17944,21825


In [765]:
# Calculate the overall passing rate

# Calculate the overall passing rate for different school size

#Calculate the total number of students in each school size range
total_students_by_type = merged_data.groupby('school_type')['Student ID'].count().reset_index(name='total_students')

# Calculate the total number of passing students (both math and reading) in each school
total_passing_by_type = merged_data.groupby('school_type').apply(
    lambda x: (x['math_pass_size'] + x['read_pass_size']).sum()).reset_index(name='total_passing_students')

# Merge the passing student counts with the total students for each size range
passing_rate_data = pd.merge(total_students_by_type, total_passing_by_type, on='school_type')

# Calculate the overall passing rate for each size range
passing_rate_data['overall_passing_rate'] = passing_rate_data['total_passing_students'] / passing_rate_data['total_students'] * 100

# Print the final data
display(passing_rate_data)

/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/4250654758.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_students_by_type = merged_data.groupby('school_type')['Student ID'].count().reset_index(name='total_students')
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/4250654758.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_passing_by_type = merged_data.groupby('school_type').apply(
/var/folders/bb/qxcf7b0d1zzbx6vt008mk2r00000gn/T/ipykernel_87099/4250654758.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is d

,school_type,total_students,total_passing_students,overall_passing_rate
0,Charter,12194,12168,99.786780
1,District,26976,25284,93.727758
